In [ ]:
!pip install newsapi-python
from newsapi import NewsApiClient
from google.colab import userdata
import requests
from datetime import datetime
import pytz #to convert time to EST
from IPython.display import Image, display

In [ ]:
#Playing around with the API, seeing what sources are provided
api_key = userdata.get('NEWS_API')
newsapi = NewsApiClient(api_key=api_key)
sources = newsapi.get_sources()
for source in sources['sources']:
        print(source['name'])

In [ ]:
def get_user_topic():
    print("Get your top news snapshot now!")
    print("Do you want to filter by:") #API does not allow category and news source filters to be used simultaneously
    print("1. Category (e.g., General, Technology, etc.)")
    print("2. Specific News Sources (e.g., BBC News, CNN, etc.)")
    choice = input("Enter 1 for Category or 2 for Sources (default is 1): ")

    topic, sources = None, None

    if choice == '2':
        print("Select Specific Sources (comma-separated, e.g., BBC News, CNN, The Wall Street Journal).")
        sources = input("Enter sources: ")
        sources_input = input("Enter sources: ")
        sources = [source.strip().lower().replace(" ", "-") for source in sources_input.split(',')]
        #if a user wants to filter on BBC News, the code must read bbc-news, creates one string of sources
    else:
        print("Select a Category:")
        print("1. General\n2. Technology\n3. Health\n4. Sports\n5. Business\n6. Entertainment")
        category_choice = input("Enter a number from 1-6 (default is General): ")

        topics = {
            '1': "general",
            '2': 'technology',
            '3': 'health',
            '4': 'sports',
            '5': 'business',
            '6': 'entertainment'
        }
        topic = topics.get(category_choice, "general")

    keyword = input("Enter a keyword to search for (optional): ") #API is a little finnicky on this part
    num_headlines = int(input("How many headlines would you like to see? (default is 5): ") or 5)

    return topic, keyword, num_headlines, sources

def show_image(image_url):
    display(Image(url=image_url,width=750, height=500)) #eventually change so pictures don't look stretched

def convert_to_est(published_at):
    utc_time = datetime.strptime(published_at, '%Y-%m-%dT%H:%M:%S%z')
    est_tz = pytz.timezone('US/Eastern')
    est_time = utc_time.astimezone(est_tz)
    return est_time.strftime("%B %d, %Y %I:%M %p EST")

def collect_news(topic, keyword, num_headlines, sources):
    if sources:
        top_headlines = newsapi.get_top_headlines(
            q=keyword or None,
            sources=','.join(sources),
            language="en"
        )
    else:
        top_headlines = newsapi.get_top_headlines(
            q=keyword or None,
            category=topic,
            language="en"
        )

    headline_number = 1
    headlines = []

    print(f"\nTop {num_headlines} Headlines for {sources or topic.capitalize()}")
    for article in top_headlines['articles']:
        if article['title'] != '[Removed]' and headline_number <= num_headlines:
            published_date = convert_to_est(article['publishedAt'])
            headline = (f"{headline_number}. {article['title']}\n"
                        f"Source: {article['source']['name']}\n"
                        f"Description: {article['description']}\n"
                        f"PPublished: {published_date}\n"
                        f"Link: {article['url']}\n")
            print(headline)
            if article.get('urlToImage'):
                    show_image(article['urlToImage'])
            headlines.append(headline)
            headline_number += 1

    if not headlines:
        print("No articles found for your query.")
    return headlines


def main():
    topic, keyword, num_headlines, sources = get_user_topic()
    headlines = collect_news(topic, keyword, num_headlines, sources)

main()

#Implement email sending here in order to save the results.


#Features to implement and think about:

Possible Items that can be achieved **NOW**:


*   Implement Email Sending Function via Mailgun
*   Resize Images so they keep original ratio, but do not exceed 1000 pixels in any direction

Powerful features that will require **additional examination**:


*   Using some sort of NLP to provide short summaries of articles
*   (1) Provide article word counts then (2) allow users to filter articles based on word count
*   Locally bookmark articles



Features that will increase user-friendliness **with web design**:


*   Use a multiselect drop down menu to select sources
*   Filter on dates and word count
*   Locally saved articles are stored in a different webpage





